In [5]:
#Setting the dependencies and kpi Key
#If you havent install citypy don forget to run pip install citpy

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from citipy import citipy

api_key="80fded64a8730f6431a58a549bf8890e"


In [11]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

628

In [12]:
#Find cities near equator 
latitude = 0
cities = []
countries = []
columns = ['City', 'Country', 'Max Temperature', 'Humidity', 'Wind Speed', 'Cloudiness', 'Longitude', 'Latitude']

for longitude in range(-180,180): 
    city = citipy.nearest_city(latitude,longitude)
    cities.append(city.city_name)
    countries.append(city.country_code)
    
equatorial_cities = {'City': cities, 'Country': countries}

equatorial_cities_df=pd.DataFrame(equatorial_cities, columns=columns).drop_duplicates().reset_index(drop=True)
equatorial_cities_df.head()

,City,Country,Max Temperature,Humidity,Wind Speed,Cloudiness,Longitude,Latitude
0,vaitupu,wf,NaN,NaN,NaN,NaN,NaN,NaN
1,saleaula,ws,NaN,NaN,NaN,NaN,NaN,NaN
2,samalaeulu,ws,NaN,NaN,NaN,NaN,NaN,NaN
3,lufilufi,ws,NaN,NaN,NaN,NaN,NaN,NaN
4,samusu,ws,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
base_url = "http://api.openweathermap.org/data/2.5/weather"
params = {"q":'', "APPID":api_key}
count=0
backup_weather_data=False

# Use iterrows to iterate through pandas dataframe
for index, row in equatorial_cities_df.iterrows():

   
    city_and_country = f"{row['City']},{row['Country']}"

    params['q'] = city_and_country
    
    try:
        response = requests.get(base_url, params=params).json()
        results  = response.get('cod')
        city     = response.get('name')
        
        if results == 200:
            sys     = response['sys']
            country = sys['country']
            weather_metrics = f"\n{city}, {country}: "
            
            main    = response['main']
            temp    = 9/5 * (float(main['temp_max']) - 273) + 32 # Convert Kelvin to Fahrenheit
            humid   = main['humidity']
            equatorial_cities_df.set_value(index, "Max Temperature", temp)
            equatorial_cities_df.set_value(index, "Humidity",        humid)
            weather_metrics += f"max temp: {temp} humidity: {humid} "
            
            wind    = response['wind']
            speed   = wind['speed']
            equatorial_cities_df.set_value(index, "Wind Speed", speed)
            weather_metrics += f"wind: {speed} "
            
            clouds  = response['clouds']
            clouds  = clouds['all']
            equatorial_cities_df.set_value(index, "Cloudiness", clouds)
            weather_metrics += f"clouds: {clouds} "
            
            coord   = response['coord']
            lon     = coord['lon']
            lat     = coord['lat']
            equatorial_cities_df.set_value(index, "Longitude", lon)
            equatorial_cities_df.set_value(index, "Latitude",  lat)
            weather_metrics += f"lon: {lon} lat: {lat} "
            
            print(weather_metrics)
            print('-' * 80)
            
        else:
            if count >= 60:
                print(response)
                backup_weather_data = True
                print('-' * 80)
                break
            count += 1            
            
    except:
        continue

C:\Users\zsantiba\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\zsantiba\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\zsantiba\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\zsantiba\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\zsantiba\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: set_v


Lufilufi, WS: max temp: 79.90160000000004 humidity: 76 wind: 8.98 clouds: 30 lon: -171.6 lat: -13.87 
--------------------------------------------------------------------------------

Faanui, PF: max temp: 77.558 humidity: 84 wind: 9.292 clouds: 61 lon: -151.75 lat: -16.48 
--------------------------------------------------------------------------------

Atuona, PF: max temp: 79.39399999999998 humidity: 72 wind: 4.791 clouds: 56 lon: -139.03 lat: -9.8 
--------------------------------------------------------------------------------

San Patricio, MX: max temp: 89.86999999999996 humidity: 70 wind: 4.1 clouds: 40 lon: -104.7 lat: 19.22 
--------------------------------------------------------------------------------

Coahuayana, MX: max temp: 78.20960000000005 humidity: 87 wind: 1 clouds: 44 lon: -100.35 lat: 18.62 
--------------------------------------------------------------------------------

Lazaro Cardenas, MX: max temp: 77.26999999999995 humidity: 50 wind: 3.6 clouds: 40 lon: -10


Buchanan, LR: max temp: 77.26999999999995 humidity: 94 wind: 6.82 clouds: 75 lon: -10.05 lat: 5.88 
--------------------------------------------------------------------------------

Harper, LR: max temp: 77.11160000000002 humidity: 86 wind: 5.21 clouds: 90 lon: -7.71 lat: 4.38 
--------------------------------------------------------------------------------

Tabou, CI: max temp: 74.01559999999998 humidity: 93 wind: 1.87 clouds: 94 lon: -7.36 lat: 4.42 
--------------------------------------------------------------------------------

Axim, GH: max temp: 77.75960000000005 humidity: 85 wind: 5.8 clouds: 99 lon: -2.24 lat: 4.87 
--------------------------------------------------------------------------------

Takoradi, GH: max temp: 74.62760000000003 humidity: 91 wind: 1.93 clouds: 98 lon: -1.75 lat: 4.89 
--------------------------------------------------------------------------------

Mumford, GH: max temp: 73.1876 humidity: 94 wind: 2.02 clouds: 100 lon: -0.76 lat: 5.26 
--------------


Loa Janan, ID: max temp: 78.69200000000001 humidity: 89 wind: 1.198 clouds: 86 lon: 117.09 lat: -0.58 
--------------------------------------------------------------------------------

Bontang, ID: max temp: 78.24560000000002 humidity: 90 wind: 1.23 clouds: 80 lon: 117.47 lat: 0.12 
--------------------------------------------------------------------------------

Palu, ID: max temp: 81.55759999999998 humidity: 73 wind: 0.75 clouds: 79 lon: 119.87 lat: -0.9 
--------------------------------------------------------------------------------

Poso, ID: max temp: 69.64159999999995 humidity: 77 wind: 1.51 clouds: 83 lon: 120.75 lat: -1.4 
--------------------------------------------------------------------------------

Gorontalo, ID: max temp: 82.9796 humidity: 68 wind: 1.82 clouds: 85 lon: 123.06 lat: 0.54 
--------------------------------------------------------------------------------

Tondano, ID: max temp: 84.46999999999997 humidity: 66 wind: 1 clouds: 20 lon: 124.91 lat: 1.3 
---------